In [1]:
import os
import sys
import itertools #important for going through the item sets and all combinations
import numpy as np
import pandas as pd
# List of all the datasets that are needed
os.listdir(os.getcwd())

['amazon_transactions.csv',
 '.ipynb_checkpoints',
 'MidtermAlgorithm.ipynb',
 'best_buy_items.csv',
 'amazon_items.csv',
 'best_buy_transactions.csv',
 'k-mart_items.csv',
 'k-mart_transactions.csv',
 'nike_items.csv',
 '.git',
 'nike_transactions.csv',
 'ace_hardware_items.csv',
 'ace_hardware_transactions.csv']

In [13]:
# Loads a dictionary of datasets that you can select by number
selected_stores = {1: "amazon", 2: "best_buy", 3: "k-mart", 4: "nike", 5: "ace_hardware"}
selected_id = int(input("Enter the store number for the dataset that you want:\n1. Amazon\n2. Best Buy\n3. K-mart\n4. Nike\n5. Ace Hardware\n"))
if selected_id not in selected_stores.keys():
    print("invalid number, There are only 5 choices!Try again next time")
    sys.exit()
item_names = pd.read_csv(f"{os.getcwd()}/{selected_stores[selected_id]}_items.csv")
transactions = pd.read_csv(f"{os.getcwd()}/{selected_stores[selected_id]}_transactions.csv")
itemset_k1 = item_names.set_index("Item Name").to_dict()["Item #"]

# Split the string by comma to seperate each string in a row
item_k1_count = transactions['Transaction'].str.split(", ").explode().value_counts()
item_k1 = item_k1_count.to_dict()
# Get the support value for each itemset
for k, _ in itemset_k1.items():
    if k not in item_k1:
        itemset_k1[k] = 0
    else:
        itemset_k1[k] = float(item_k1[k]) / len(transactions["Transaction"])
        
print(item_k1_count)
print(itemset_k1)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


Enter the store number for the dataset that you want:
1. Amazon
2. Best Buy
3. K-mart
4. Nike
5. Ace Hardware
 1


{'A Beginner’s Guide': 1, 'Java: The Complete Reference': 2, 'Java For Dummies': 3, 'Android Programming: The Big Nerd Ranch': 4, 'Head First Java 2nd Edition': 5, 'Beginning Programming with Java': 6, 'Java 8 Pocket Guide': 7, 'C++ Programming in Easy Steps': 8, 'Effective Java (2nd Edition)': 9, 'HTML and CSS: Design and Build Websites': 10}
2
Transaction
Java For Dummies                           13
Android Programming: The Big Nerd Ranch    13
A Beginner’s Guide                         11
Java: The Complete Reference               10
Head First Java 2nd Edition                 8
Beginning Programming with Java             6
Java 8 Pocket Guide                         4
HTML and CSS: Design and Build Websites     2
C++ Programming in Easy Steps               1
Name: count, dtype: int64
{'A Beginner’s Guide': 0.55, 'Java: The Complete Reference': 0.5, 'Java For Dummies': 0.65, 'Android Programming: The Big Nerd Ranch': 0.65, 'Head First Java 2nd Edition': 0.4, 'Beginning Programming 